In [1]:
# Import the modules
import cv2
from skimage.feature import hog
from sklearn import datasets
import numpy as np
import joblib


In [5]:
# Load the classifier
clf, pp = joblib.load("digits_cls1.pkl")

In [2]:
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# Dummy initializations (replace with your actual preprocessor and classifier)
pp = StandardScaler()  # Initialize your preprocessor
clf = SVC()  # Initialize your classifier (should be trained)

def main():
    # Open Camera
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Camera could not be opened.")
        return

    while cap.isOpened():
        # Capture frames from the camera
        ret, img = cap.read()
        if not ret:
            print("Failed to capture image")
            break

        # Apply get_img_contour_thresh function on frame
        img, contours, thresh = get_img_contour_thresh(img)
        ans = ''

        if len(contours) > 0:
            # Find contour with maximum area
            contour = max(contours, key=cv2.contourArea)
            # Ranging contourArea
            if 1500 < cv2.contourArea(contour) < 5000:
                # Dimensions of rectangle
                x, y, w, h = cv2.boundingRect(contour)
                # Making new image containing contour for classification
                newImage = thresh[y:y + h, x:x + w]
                # Resize new image
                newImage = cv2.resize(newImage, (28, 28))
                newImage = np.array(newImage)

                # Perform classification
                hog_ft = hog(newImage, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualize=False)
                hog_ft = pp.transform(np.array([hog_ft], dtype='float64'))
                ans = clf.predict(hog_ft)
                
        # Draw rectangle and put text
        x, y, w, h = 0, 0, 300, 300
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(img, "SVM: " + str(ans), (10, 320), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Show frame and threshold
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)

        k = cv2.waitKey(10)
        if k == 27:  # ESC key to exit
            break

    # Release the camera and destroy windows
    cap.release()
    cv2.destroyAllWindows()

def get_img_contour_thresh(img):
    x, y, w, h = 0, 0, 300, 300
    
    # Change color-space from BGR -> Gray
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian Blur and Threshold
    blur = cv2.GaussianBlur(gray, (35, 35), 0)
    ret, thresh = cv2.threshold(blur, 70, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Making thresh image of size x, y, w, h = 0, 0, 300, 300
    thresh = thresh[y:y + h, x:x + w]
    
    # Find contours
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    
    return img, contours, thresh

if __name__ == "__main__":
    main()


NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [7]:
main()

TypeError: hog() got an unexpected keyword argument 'visualise'

: 